In [1]:
import numpy as np
import pickle

import os
import sys
sys.path.append('../Parameters/')
sys.path.append('../AnalyticalModel/')

from Metadata import read_metadata
import ParametersFromLiterature as prms

from FullMatrix import FullStates
from AnalyticalSimulationRun import get_steady_states, get_means, get_means_and_instantaneous_vars, get_means_and_vars, get_correlation_times

In [2]:
def read_files_shift_compensation(file):
    shift_compensation = None
    try:
        f = open(file+'_shift_compensation', 'rb')
        shift_compensation = pickle.load(f)
        f.close()
    except:
        print('Shift compensation could not be loaded.')
    return shift_compensation
        
def read_files_steady_state(file, grid_prms, varied_prms):
    steady_states      = None
    probabilities      = None
    try:
        f = open(file+'_steady_states', 'rb')
        steady_states = pickle.load(f)
        f.close()

        probabilities = []
        S = FullStates(grid_prms.L)
        sites = grid_prms.sites.flatten()
        for J, eb in zip(varied_prms.J_list, varied_prms.eb_list):
            probabilities.append([])
            for htf in varied_prms.htf_list:
                probabilities[-1].append(S.compact_states(steady_states[f'J{J}eb{eb}htf{htf}'][1], sites))
        probabilities = np.array(probabilities)
    except:
        print('Steady states could not be loaded.')
    return steady_states, probabilities

def read_files_means(file, varied_prms):
    means_dict         = None
    means              = None
    try:
        f = open(file+'_means', 'rb')
        means_dict = pickle.load(f)
        f.close()

        means = []
        for J, eb in zip(varied_prms.J_list, varied_prms.eb_list):
            means.append([])
            for htf in varied_prms.htf_list:
                means[-1].append(means_dict[f'J{J}eb{eb}htf{htf}'])
        means = np.array(means)
    except:
        print('Means could not be loaded.')
    return means_dict, means

def read_files_means_and_vars(file, varied_prms):
    means_and_vars     = None
    means              = None
    stds               = None
    try:
        f = open(file+'_means_and_vars', 'rb')
        means_and_vars = pickle.load(f)
        f.close()

        means = []
        stds  = []
        for J, eb in zip(varied_prms.J_list, varied_prms.eb_list):
            means.append([])
            stds.append([])
            for htf in varied_prms.htf_list:
                mu, sigma_squared = means_and_vars[f'J{J}eb{eb}htf{htf}']
                means[-1].append(mu)
                stds[-1].append(np.sqrt(sigma_squared))
        means = np.array(means)
        stds  = np.array(stds)
    except:
        print('Means and vars could not be loaded.')
    return means_and_vars, means, stds

def read_files_correlation_times(file, varied_prms):
    correlation_times  = None
    taus               = None
    try:
        f = open(file+'_correlation_times', 'rb')
        correlation_times = pickle.load(f)
        f.close()

        taus = []
        for J, eb in zip(varied_prms.J_list, varied_prms.eb_list):
            taus.append([])
            for htf in varied_prms.htf_list:
                taus[-1].append(correlation_times[f'J{J}eb{eb}htf{htf}'])
        taus = np.array(taus)
    except:
        print('Correlation times could not be loaded.')
    return correlation_times, taus

def read_files(file): 
    grid_prms, sim_prms, varied_prms = read_metadata(file)
    shift_compensation               = read_files_shift_compensation(file)
    steady_states, probabilities     = read_files_steady_state(file, grid_prms, varied_prms)
    means_and_vars, means, stds      = read_files_means_and_vars(file, varied_prms)
    correlation_times, taus          = read_files_correlation_times(file, varied_prms)
    
    means_dict = None
    if np.any(means == None):
        means_dict, means            = read_files_means(file, varied_prms)
        
    return grid_prms, sim_prms, varied_prms, shift_compensation, steady_states, probabilities, means_dict, \
           means, means_and_vars, stds, correlation_times, taus 

*Read files:*

In [4]:
file_names = ['Measurement/6Sites_NoPadding_NonPeriodic_SteadyState_InfiniteBath_Occupancy_2024-04-30',\
              'Measurement/6Sites_2Padding_NonPeriodic_SteadyState_InfiniteBath_Occupancy_2024-04-30',\
              'Measurement/6Sites_NoPadding_NonPeriodic_SteadyState_InfiniteBath_N6Expression_2024-04-30',\
              'Measurement/6Sites_2Padding_NonPeriodic_SteadyState_InfiniteBath_N6Expression_2024-04-30',\
              'Measurement/6Sites_NoPadding_NonPeriodic_SteadyState_InfiniteBath_Occupancy_10s_2025-09-11',\
              '6Sites_NoPadding_NonPeriodic_SteadyState_InfiniteBath_Occupancy_2025-05-09',\
              '6Sites_2Padding_NonPeriodic_SteadyState_InfiniteBath_Occupancy_2025-05-09']

## Choose file to load:
file_name = file_names[4]
file = os.path.join(os.path.realpath('__file__'), f'../../Data/{file_name}')
print(f"Loading {file_name}")

grid_prms, sim_prms, varied_prms, shift_compensation, steady_states, probabilities, means_dict, \
means, means_and_vars, stds, correlation_times, taus = read_files(file)

Loading Measurement/6Sites_NoPadding_NonPeriodic_SteadyState_InfiniteBath_Occupancy_10s_2025-09-11
Correlation times could not be loaded.


<ipython-input-2-0ffe91b12ee2>:66: RuntimeWarning: invalid value encountered in sqrt
  stds[-1].append(np.sqrt(sigma_squared))


*Create files:*

In [4]:
get_steady_states(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, file=file)

J=0.0, eb=14.143319868324545, htf=-19.172368036976454
J=0.0, eb=14.143319868324545, htf=-18.073755738876944
J=0.0, eb=14.143319868324545, htf=-17.56293010567955
J=0.0, eb=14.143319868324545, htf=-17.22645785962693
J=0.0, eb=14.143319868324545, htf=-16.97514342191462
J=0.0, eb=14.143319868324545, htf=-16.774472717021062
J=0.0, eb=14.143319868324545, htf=-16.60741862292649
J=0.0, eb=14.143319868324545, htf=-16.464317769854414
J=0.0, eb=14.143319868324545, htf=-16.339154617469003
J=0.0, eb=14.143319868324545, htf=-16.227928972927376
J=0.0, eb=14.143319868324545, htf=-16.12784550493899
J=0.0, eb=14.143319868324545, htf=-16.036873717301855
J=0.0, eb=14.143319868324545, htf=-15.9534920989314
J=0.0, eb=14.143319868324545, htf=-15.876531048363866
J=0.0, eb=14.143319868324545, htf=-15.805072074950317
J=0.0, eb=14.143319868324545, htf=-15.73838069102024
J=0.0, eb=14.143319868324545, htf=-15.6758603246075
J=0.0, eb=14.143319868324545, htf=-15.61701981515316
J=0.0, eb=14.143319868324545, htf=-15.5

J=0.0, eb=14.143319868324545, htf=-13.400925404820649
J=0.0, eb=14.143319868324545, htf=-13.39471419529659
J=0.0, eb=14.143319868324545, htf=-13.38854132675809
J=0.0, eb=14.143319868324545, htf=-13.382406328759156
J=0.0, eb=14.143319868324545, htf=-13.376308739459619
J=0.0, eb=14.143319868324545, htf=-13.37024810541651
J=0.0, eb=14.143319868324545, htf=-13.36422398138171
J=0.0, eb=14.143319868324545, htf=-13.358235930105668
J=0.0, eb=14.143319868324545, htf=-13.352283522146953
J=0.0, eb=14.143319868324545, htf=-13.346366335687446
J=0.0, eb=14.143319868324545, htf=-13.34048395635296
J=0.0, eb=14.143319868324545, htf=-13.334635977039119
J=0.0, eb=14.143319868324545, htf=-13.32882199774228
J=0.0, eb=14.143319868324545, htf=-13.32304162539536
J=0.0, eb=14.143319868324545, htf=-13.317294473708372
J=0.0, eb=14.143319868324545, htf=-13.311580163013515
J=0.0, eb=14.143319868324545, htf=-13.305898320114663
J=0.0, eb=14.143319868324545, htf=-13.300248578141124
J=0.0, eb=14.143319868324545, htf=-

J=0.0, eb=14.143319868324545, htf=-12.712460573976495
J=0.0, eb=14.143319868324545, htf=-12.709335562001927
J=0.0, eb=14.143319868324545, htf=-12.706220285253544
J=0.0, eb=14.143319868324545, htf=-12.703114683263959
J=0.0, eb=14.143319868324545, htf=-12.700018696127398
J=0.0, eb=14.143319868324545, htf=-12.696932264492775
J=0.0, eb=14.143319868324545, htf=-12.693855329556863
J=0.0, eb=14.143319868324545, htf=-12.690787833057568
J=0.0, eb=14.143319868324545, htf=-12.687729717267315
J=0.0, eb=14.143319868324545, htf=-12.684680924986523
J=0.0, eb=14.143319868324545, htf=-12.681641399537192
J=0.0, eb=14.143319868324545, htf=-12.67861108475658
J=0.0, eb=14.143319868324545, htf=-12.67558992499097
J=0.0, eb=14.143319868324545, htf=-12.672577865089545
J=0.0, eb=14.143319868324545, htf=-12.669574850398341
J=0.0, eb=14.143319868324545, htf=-12.666580826754304
J=0.0, eb=14.143319868324545, htf=-12.663595740479417
J=0.0, eb=14.143319868324545, htf=-12.660619538374936
J=0.0, eb=14.143319868324545, 

J=0.0, eb=14.143319868324545, htf=-12.312748643963834
J=0.0, eb=14.143319868324545, htf=-12.31065219770586
J=0.0, eb=14.143319868324545, htf=-12.30856013730219
J=0.0, eb=14.143319868324545, htf=-12.306472444440258
J=0.0, eb=14.143319868324545, htf=-12.304389100921957
J=0.0, eb=14.143319868324545, htf=-12.30231008866268
J=0.0, eb=14.143319868324545, htf=-12.300235389690371
J=0.0, eb=14.143319868324545, htf=-12.298164986144615
J=0.0, eb=14.143319868324545, htf=-12.296098860275697
J=0.0, eb=14.143319868324545, htf=-12.29403699444369
J=0.0, eb=14.143319868324545, htf=-12.291979371117563
J=0.0, eb=14.143319868324545, htf=-12.289925972874274
J=0.0, eb=14.143319868324545, htf=-12.287876782397891
J=0.0, eb=14.143319868324545, htf=-12.285831782478716
J=0.0, eb=14.143319868324545, htf=-12.283790956012416
J=0.0, eb=14.143319868324545, htf=-12.281754285999165
J=0.0, eb=14.143319868324545, htf=-12.279721755542795
J=0.0, eb=14.143319868324545, htf=-12.277693347849954
J=0.0, eb=14.143319868324545, ht

J=0.0, eb=14.143319868324545, htf=-12.023230440028065
J=0.0, eb=14.143319868324545, htf=-12.021660571561483
J=0.0, eb=14.143319868324545, htf=-12.02009316369001
J=0.0, eb=14.143319868324545, htf=-12.018528208712246
J=0.0, eb=14.143319868324545, htf=-12.016965698962894
J=0.0, eb=14.143319868324545, htf=-12.01540562681253
J=0.0, eb=14.143319868324545, htf=-12.013847984667386
J=0.0, eb=14.143319868324545, htf=-12.012292764969118
J=0.0, eb=14.143319868324545, htf=-12.010739960194597
J=0.0, eb=14.143319868324545, htf=-12.00918956285568
J=0.0, eb=14.143319868324545, htf=-12.007641565499004
J=1.0, eb=13.289575767386253, htf=-19.172368036976454
J=1.0, eb=13.289575767386253, htf=-18.073755738876944
J=1.0, eb=13.289575767386253, htf=-17.56293010567955
J=1.0, eb=13.289575767386253, htf=-17.22645785962693
J=1.0, eb=13.289575767386253, htf=-16.97514342191462
J=1.0, eb=13.289575767386253, htf=-16.774472717021062
J=1.0, eb=13.289575767386253, htf=-16.60741862292649
J=1.0, eb=13.289575767386253, htf=-

...

J=14.0, eb=2.143424269595327, htf=-13.36422398138171
J=14.0, eb=2.143424269595327, htf=-13.358235930105668
J=14.0, eb=2.143424269595327, htf=-13.352283522146953
J=14.0, eb=2.143424269595327, htf=-13.346366335687446
J=14.0, eb=2.143424269595327, htf=-13.34048395635296
J=14.0, eb=2.143424269595327, htf=-13.334635977039119
J=14.0, eb=2.143424269595327, htf=-13.32882199774228
J=14.0, eb=2.143424269595327, htf=-13.32304162539536
J=14.0, eb=2.143424269595327, htf=-13.317294473708372
J=14.0, eb=2.143424269595327, htf=-13.311580163013515
J=14.0, eb=2.143424269595327, htf=-13.305898320114663
J=14.0, eb=2.143424269595327, htf=-13.300248578141124
J=14.0, eb=2.143424269595327, htf=-13.294630576405481
J=14.0, eb=2.143424269595327, htf=-13.289043960265422
J=14.0, eb=2.143424269595327, htf=-13.2834883809894
J=14.0, eb=2.143424269595327, htf=-13.27796349562601
J=14.0, eb=2.143424269595327, htf=-13.272468966876948
J=14.0, eb=2.143424269595327, htf=-13.26700446297345
J=14.0, eb=2.143424269595327, htf=-1

J=14.0, eb=2.143424269595327, htf=-12.696932264492775
J=14.0, eb=2.143424269595327, htf=-12.693855329556863
J=14.0, eb=2.143424269595327, htf=-12.690787833057568
J=14.0, eb=2.143424269595327, htf=-12.687729717267315
J=14.0, eb=2.143424269595327, htf=-12.684680924986523
J=14.0, eb=2.143424269595327, htf=-12.681641399537192
J=14.0, eb=2.143424269595327, htf=-12.67861108475658
J=14.0, eb=2.143424269595327, htf=-12.67558992499097
J=14.0, eb=2.143424269595327, htf=-12.672577865089545
J=14.0, eb=2.143424269595327, htf=-12.669574850398341
J=14.0, eb=2.143424269595327, htf=-12.666580826754304
J=14.0, eb=2.143424269595327, htf=-12.663595740479417
J=14.0, eb=2.143424269595327, htf=-12.660619538374936
J=14.0, eb=2.143424269595327, htf=-12.6576521677157
J=14.0, eb=2.143424269595327, htf=-12.65469357624452
J=14.0, eb=2.143424269595327, htf=-12.651743712166667
J=14.0, eb=2.143424269595327, htf=-12.648802524144415
J=14.0, eb=2.143424269595327, htf=-12.645869961291703
J=14.0, eb=2.143424269595327, htf

J=14.0, eb=2.143424269595327, htf=-12.30856013730219
J=14.0, eb=2.143424269595327, htf=-12.306472444440258
J=14.0, eb=2.143424269595327, htf=-12.304389100921957
J=14.0, eb=2.143424269595327, htf=-12.30231008866268
J=14.0, eb=2.143424269595327, htf=-12.300235389690371
J=14.0, eb=2.143424269595327, htf=-12.298164986144615
J=14.0, eb=2.143424269595327, htf=-12.296098860275697
J=14.0, eb=2.143424269595327, htf=-12.29403699444369
J=14.0, eb=2.143424269595327, htf=-12.291979371117563
J=14.0, eb=2.143424269595327, htf=-12.289925972874274
J=14.0, eb=2.143424269595327, htf=-12.287876782397891
J=14.0, eb=2.143424269595327, htf=-12.285831782478716
J=14.0, eb=2.143424269595327, htf=-12.283790956012416
J=14.0, eb=2.143424269595327, htf=-12.281754285999165
J=14.0, eb=2.143424269595327, htf=-12.279721755542795
J=14.0, eb=2.143424269595327, htf=-12.277693347849954
J=14.0, eb=2.143424269595327, htf=-12.275669046229275
J=14.0, eb=2.143424269595327, htf=-12.273648834090551
J=14.0, eb=2.143424269595327, h

J=14.0, eb=2.143424269595327, htf=-12.031116963392282
J=14.0, eb=2.143424269595327, htf=-12.029534675149693
J=14.0, eb=2.143424269595327, htf=-12.027954886558714
J=14.0, eb=2.143424269595327, htf=-12.026377589734006
J=14.0, eb=2.143424269595327, htf=-12.024802776827478
J=14.0, eb=2.143424269595327, htf=-12.023230440028065
J=14.0, eb=2.143424269595327, htf=-12.021660571561483
J=14.0, eb=2.143424269595327, htf=-12.02009316369001
J=14.0, eb=2.143424269595327, htf=-12.018528208712246
J=14.0, eb=2.143424269595327, htf=-12.016965698962894
J=14.0, eb=2.143424269595327, htf=-12.01540562681253
J=14.0, eb=2.143424269595327, htf=-12.013847984667386
J=14.0, eb=2.143424269595327, htf=-12.012292764969118
J=14.0, eb=2.143424269595327, htf=-12.010739960194597
J=14.0, eb=2.143424269595327, htf=-12.00918956285568
J=14.0, eb=2.143424269595327, htf=-12.007641565499004


{'J0.0eb14.143319868324545htf-19.172368036976454': (array(['0000000000', '0000000001', '0000000010', ..., '1111111101',
         '1111111110', '1111111111'], dtype='<U10'),
  array([9.61613873e-01, 4.53468450e-09, 4.53468450e-09, ...,
         7.92706787e-39, 7.92706787e-39, 3.73816901e-47])),
 'J0.0eb14.143319868324545htf-18.073755738876944': (array(['0000000000', '0000000001', '0000000010', ..., '1111111101',
         '1111111110', '1111111111'], dtype='<U10'),
  array([8.89879661e-01, 1.25892221e-08, 1.25892221e-08, ...,
         1.44389119e-34, 1.44389119e-34, 2.04268821e-42])),
 'J0.0eb14.143319868324545htf-17.56293010567955': (array(['0000000000', '0000000001', '0000000010', ..., '1111111101',
         '1111111110', '1111111111'], dtype='<U10'),
  array([8.24311470e-01, 1.94360368e-08, 1.94360368e-08, ...,
         1.32719065e-32, 1.32719065e-32, 3.12931788e-40])),
 'J0.0eb14.143319868324545htf-17.22645785962693': (array(['0000000000', '0000000001', '0000000010', ..., '1111111101

In [ ]:
get_means(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, sim_prms.T, N=shift_compensation.N_expr, file=file)

In [5]:
get_means_and_instantaneous_vars(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, sim_prms.T, N=shift_compensation.N_expr, file=file)

J=0.0, eb=14.143319868324545, htf=-16.86978290154109
J=0.0, eb=14.143319868324545, htf=-15.771170518558932
J=0.0, eb=14.143319868324545, htf=-15.260344800478883
J=0.0, eb=14.143319868324545, htf=-14.923872469543603
J=0.0, eb=14.143319868324545, htf=-14.672557946948622
J=0.0, eb=14.143319868324545, htf=-14.471887157172386
J=0.0, eb=14.143319868324545, htf=-14.304832978195126
J=0.0, eb=14.143319868324545, htf=-14.161732040240349
J=0.0, eb=14.143319868324545, htf=-14.036568802972232
J=0.0, eb=14.143319868324545, htf=-13.925343073547888
J=0.0, eb=14.143319868324545, htf=-13.825259520676775
J=0.0, eb=14.143319868324545, htf=-13.734287648156911
J=0.0, eb=14.143319868324545, htf=-13.650905944903712
J=0.0, eb=14.143319868324545, htf=-13.573944809453428
J=0.0, eb=14.143319868324545, htf=-13.502485751157119
J=0.0, eb=14.143319868324545, htf=-13.435794282344272
J=0.0, eb=14.143319868324545, htf=-13.373273831048756
J=0.0, eb=14.143319868324545, htf=-13.314433236711631
J=0.0, eb=14.143319868324545,

J=2.0, eb=12.421156590900992, htf=-12.726645251409895
J=2.0, eb=12.421156590900992, htf=-12.695392613591466
J=2.0, eb=12.421156590900992, htf=-12.665087169781803
J=2.0, eb=12.421156590900992, htf=-12.635673190261167
J=2.0, eb=12.421156590900992, htf=-12.60709972350276
J=2.0, eb=12.421156590900992, htf=-12.579320065081323
J=2.0, eb=12.421156590900992, htf=-12.552291298379034
J=2.0, eb=12.421156590900992, htf=-12.525973895747283
J=2.0, eb=12.421156590900992, htf=-12.500331370819557
J=2.0, eb=12.421156590900992, htf=-12.475329974299745
J=2.0, eb=12.421156590900992, htf=-12.45093842686118
J=2.0, eb=12.421156590900992, htf=-12.427127683853048
J=2.0, eb=12.421156590900992, htf=-12.403870727374358
J=2.0, eb=12.421156590900992, htf=-12.38114238198237
J=2.0, eb=12.421156590900992, htf=-12.358919150883219
J=2.0, eb=12.421156590900992, htf=-12.337179069932363
J=2.0, eb=12.421156590900992, htf=-12.31590157717062
J=2.0, eb=12.421156590900992, htf=-12.29506739595331
J=2.0, eb=12.421156590900992, htf

J=5.0, eb=9.890137678918109, htf=-12.665087169781803
J=5.0, eb=9.890137678918109, htf=-12.635673190261167
J=5.0, eb=9.890137678918109, htf=-12.60709972350276
J=5.0, eb=9.890137678918109, htf=-12.579320065081323
J=5.0, eb=9.890137678918109, htf=-12.552291298379034
J=5.0, eb=9.890137678918109, htf=-12.525973895747283
J=5.0, eb=9.890137678918109, htf=-12.500331370819557
J=5.0, eb=9.890137678918109, htf=-12.475329974299745
J=5.0, eb=9.890137678918109, htf=-12.45093842686118
J=5.0, eb=9.890137678918109, htf=-12.427127683853048
J=5.0, eb=9.890137678918109, htf=-12.403870727374358
J=5.0, eb=9.890137678918109, htf=-12.38114238198237
J=5.0, eb=9.890137678918109, htf=-12.358919150883219
J=5.0, eb=9.890137678918109, htf=-12.337179069932363
J=5.0, eb=9.890137678918109, htf=-12.31590157717062
J=5.0, eb=9.890137678918109, htf=-12.29506739595331
J=5.0, eb=9.890137678918109, htf=-12.274658430007628
J=5.0, eb=9.890137678918109, htf=-12.254657668986473
J=5.0, eb=9.890137678918109, htf=-12.23504910328360

J=8.0, eb=7.416092589646984, htf=-13.01963313060565
J=8.0, eb=7.416092589646984, htf=-12.977960339890828
J=8.0, eb=7.416092589646984, htf=-12.937954910962866
J=8.0, eb=7.416092589646984, htf=-12.899488535820797
J=8.0, eb=7.416092589646984, htf=-12.862447169826169
J=8.0, eb=7.416092589646984, htf=-12.8267289929098
J=8.0, eb=7.416092589646984, htf=-12.792242722524332
J=8.0, eb=7.416092589646984, htf=-12.758906207942433
J=8.0, eb=7.416092589646984, htf=-12.726645251409895
J=8.0, eb=7.416092589646984, htf=-12.695392613591466
J=8.0, eb=7.416092589646984, htf=-12.665087169781803
J=8.0, eb=7.416092589646984, htf=-12.635673190261167
J=8.0, eb=7.416092589646984, htf=-12.60709972350276
J=8.0, eb=7.416092589646984, htf=-12.579320065081323
J=8.0, eb=7.416092589646984, htf=-12.552291298379034
J=8.0, eb=7.416092589646984, htf=-12.525973895747283
J=8.0, eb=7.416092589646984, htf=-12.500331370819557
J=8.0, eb=7.416092589646984, htf=-12.475329974299745
J=8.0, eb=7.416092589646984, htf=-12.4509384268611

J=11.0, eb=4.940321020899742, htf=-13.825259520676775
J=11.0, eb=4.940321020899742, htf=-13.734287648156911
J=11.0, eb=4.940321020899742, htf=-13.650905944903712
J=11.0, eb=4.940321020899742, htf=-13.573944809453428
J=11.0, eb=4.940321020899742, htf=-13.502485751157119
J=11.0, eb=4.940321020899742, htf=-13.435794282344272
J=11.0, eb=4.940321020899742, htf=-13.373273831048756
J=11.0, eb=4.940321020899742, htf=-13.314433236711631
J=11.0, eb=4.940321020899742, htf=-13.25886329124262
J=11.0, eb=4.940321020899742, htf=-13.206219463442988
J=11.0, eb=4.940321020899742, htf=-13.156208948554108
J=11.0, eb=4.940321020899742, htf=-13.108580805250627
J=11.0, eb=4.940321020899742, htf=-13.063118336859633
J=11.0, eb=4.940321020899742, htf=-13.01963313060565
J=11.0, eb=4.940321020899742, htf=-12.977960339890828
J=11.0, eb=4.940321020899742, htf=-12.937954910962866
J=11.0, eb=4.940321020899742, htf=-12.899488535820797
J=11.0, eb=4.940321020899742, htf=-12.862447169826169
J=11.0, eb=4.940321020899742, 

J=13.0, eb=3.284627203306573, htf=-12.025589873280024
J=13.0, eb=3.284627203306573, htf=-12.009964461062335
J=14.0, eb=2.4541905752958177, htf=-16.86978290154109
J=14.0, eb=2.4541905752958177, htf=-15.771170518558932
J=14.0, eb=2.4541905752958177, htf=-15.260344800478883
J=14.0, eb=2.4541905752958177, htf=-14.923872469543603
J=14.0, eb=2.4541905752958177, htf=-14.672557946948622
J=14.0, eb=2.4541905752958177, htf=-14.471887157172386
J=14.0, eb=2.4541905752958177, htf=-14.304832978195126
J=14.0, eb=2.4541905752958177, htf=-14.161732040240349
J=14.0, eb=2.4541905752958177, htf=-14.036568802972232
J=14.0, eb=2.4541905752958177, htf=-13.925343073547888
J=14.0, eb=2.4541905752958177, htf=-13.825259520676775
J=14.0, eb=2.4541905752958177, htf=-13.734287648156911
J=14.0, eb=2.4541905752958177, htf=-13.650905944903712
J=14.0, eb=2.4541905752958177, htf=-13.573944809453428
J=14.0, eb=2.4541905752958177, htf=-13.502485751157119
J=14.0, eb=2.4541905752958177, htf=-13.435794282344272
J=14.0, eb=2.

{'J0.0eb14.143319868324545htf-16.86978290154109': (0.06142977467703202,
  0.009609359576693514),
 'J0.0eb14.143319868324545htf-15.771170518558932': (0.1641250133781329,
  0.02286466556029343),
 'J0.0eb14.143319868324545htf-15.260344800478883': (0.24656354498817012,
  0.030961660545172777),
 'J0.0eb14.143319868324545htf-14.923872469543603': (0.31420080006916873,
  0.035913109550843855),
 'J0.0eb14.143319868324545htf-14.672557946948622': (0.37069461162195033,
  0.03888001942273367),
 'J0.0eb14.143319868324545htf-14.471887157172386': (0.41858926463077245,
  0.040562048694440245),
 'J0.0eb14.143319868324545htf-14.304832978195126': (0.4597092714314571,
  0.04139610953190274),
 'J0.0eb14.143319868324545htf-14.161732040240349': (0.4953970870557022,
  0.04166313553207118),
 'J0.0eb14.143319868324545htf-14.036568802972232': (0.5266624511713587,
  0.041548185616255826),
 'J0.0eb14.143319868324545htf-13.925343073547888': (0.5542794497434439,
  0.041175623555924856),
 'J0.0eb14.143319868324545htf-

In [5]:
get_means_and_vars(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, sim_prms.T, N=shift_compensation.N_expr, file=file)

J=0.0, eb=14.143319868324545, htf=-16.86978290154109
J=0.0, eb=14.143319868324545, htf=-15.771170518558932
J=0.0, eb=14.143319868324545, htf=-15.260344800478883
J=0.0, eb=14.143319868324545, htf=-14.923872469543603
J=0.0, eb=14.143319868324545, htf=-14.672557946948622
J=0.0, eb=14.143319868324545, htf=-14.471887157172386
J=0.0, eb=14.143319868324545, htf=-14.304832978195126
J=0.0, eb=14.143319868324545, htf=-14.161732040240349
J=0.0, eb=14.143319868324545, htf=-14.036568802972232
J=0.0, eb=14.143319868324545, htf=-13.925343073547888
J=0.0, eb=14.143319868324545, htf=-13.825259520676775
J=0.0, eb=14.143319868324545, htf=-13.734287648156911
J=0.0, eb=14.143319868324545, htf=-13.650905944903712
J=0.0, eb=14.143319868324545, htf=-13.573944809453428
J=0.0, eb=14.143319868324545, htf=-13.502485751157119
J=0.0, eb=14.143319868324545, htf=-13.435794282344272
J=0.0, eb=14.143319868324545, htf=-13.373273831048756
J=0.0, eb=14.143319868324545, htf=-13.314433236711631
J=0.0, eb=14.143319868324545,

J=2.0, eb=12.421156590900992, htf=-12.475329974299745
J=2.0, eb=12.421156590900992, htf=-12.45093842686118
J=2.0, eb=12.421156590900992, htf=-12.427127683853048
J=2.0, eb=12.421156590900992, htf=-12.403870727374358
J=2.0, eb=12.421156590900992, htf=-12.38114238198237
J=2.0, eb=12.421156590900992, htf=-12.358919150883219
J=2.0, eb=12.421156590900992, htf=-12.337179069932363
J=2.0, eb=12.421156590900992, htf=-12.31590157717062
J=2.0, eb=12.421156590900992, htf=-12.29506739595331
J=2.0, eb=12.421156590900992, htf=-12.274658430007628
J=2.0, eb=12.421156590900992, htf=-12.254657668986473
J=2.0, eb=12.421156590900992, htf=-12.235049103283602
J=2.0, eb=12.421156590900992, htf=-12.215817647041211
J=2.0, eb=12.421156590900992, htf=-12.196949068422317
J=2.0, eb=12.421156590900992, htf=-12.17842992634056
J=2.0, eb=12.421156590900992, htf=-12.160247512942838
J=2.0, eb=12.421156590900992, htf=-12.142389801228294
J=2.0, eb=12.421156590900992, htf=-12.124845397262838
J=2.0, eb=12.421156590900992, htf

J=5.0, eb=9.890137678918109, htf=-15.771170518558932
J=5.0, eb=9.890137678918109, htf=-15.260344800478883
J=5.0, eb=9.890137678918109, htf=-14.923872469543603
J=5.0, eb=9.890137678918109, htf=-14.672557946948622
J=5.0, eb=9.890137678918109, htf=-14.471887157172386
J=5.0, eb=9.890137678918109, htf=-14.304832978195126
J=5.0, eb=9.890137678918109, htf=-14.161732040240349
J=5.0, eb=9.890137678918109, htf=-14.036568802972232
J=5.0, eb=9.890137678918109, htf=-13.925343073547888
J=5.0, eb=9.890137678918109, htf=-13.825259520676775
J=5.0, eb=9.890137678918109, htf=-13.734287648156911
J=5.0, eb=9.890137678918109, htf=-13.650905944903712
J=5.0, eb=9.890137678918109, htf=-13.573944809453428
J=5.0, eb=9.890137678918109, htf=-13.502485751157119
J=5.0, eb=9.890137678918109, htf=-13.435794282344272
J=5.0, eb=9.890137678918109, htf=-13.373273831048756
J=5.0, eb=9.890137678918109, htf=-13.314433236711631
J=5.0, eb=9.890137678918109, htf=-13.25886329124262
J=5.0, eb=9.890137678918109, htf=-13.2062194634

J=7.0, eb=8.23962329975328, htf=-12.862447169826169
J=7.0, eb=8.23962329975328, htf=-12.8267289929098
J=7.0, eb=8.23962329975328, htf=-12.792242722524332
J=7.0, eb=8.23962329975328, htf=-12.758906207942433
J=7.0, eb=8.23962329975328, htf=-12.726645251409895
J=7.0, eb=8.23962329975328, htf=-12.695392613591466
J=7.0, eb=8.23962329975328, htf=-12.665087169781803
J=7.0, eb=8.23962329975328, htf=-12.635673190261167
J=7.0, eb=8.23962329975328, htf=-12.60709972350276
J=7.0, eb=8.23962329975328, htf=-12.579320065081323
J=7.0, eb=8.23962329975328, htf=-12.552291298379034
J=7.0, eb=8.23962329975328, htf=-12.525973895747283
J=7.0, eb=8.23962329975328, htf=-12.500331370819557
J=7.0, eb=8.23962329975328, htf=-12.475329974299745
J=7.0, eb=8.23962329975328, htf=-12.45093842686118
J=7.0, eb=8.23962329975328, htf=-12.427127683853048
J=7.0, eb=8.23962329975328, htf=-12.403870727374358
J=7.0, eb=8.23962329975328, htf=-12.38114238198237
J=7.0, eb=8.23962329975328, htf=-12.358919150883219
J=7.0, eb=8.23962

J=10.0, eb=5.767304689958269, htf=-13.925343073547888
variance: -0.1486485722244628
integral: [[-4.77306592e+00 -9.66370952e+00 -1.16282232e+01 ... -5.40371209e-01
  -6.18578672e-01  2.00502944e+00]
 [-2.78378170e-03  4.94542408e+00 -3.31452255e-03 ... -1.35692619e-04
  -1.77208418e-04  5.74070637e-04]
 [-2.78308293e-03 -2.70567965e-03  4.94488774e+00 ... -1.50494263e-04
  -1.77246129e-04  5.73819922e-04]
 ...
 [ 2.20241564e-05  1.92700322e-05  3.04370824e-05 ...  4.94822673e+00
  -2.22318568e-05 -3.49120248e-05]
 [ 5.38985722e-01  5.45113713e-01  7.10738085e-01 ... -5.11139296e-01
   4.50802530e+00 -7.87387725e-01]
 [ 3.45572698e+00  3.41212001e+00  4.79782894e+00 ... -3.18088141e+00
  -3.10590748e+00  1.35207027e-02]]
J=10.0, eb=5.767304689958269, htf=-13.825259520676775
variance: -0.010452616583422761
integral: [[-5.42231798e-01 -2.94416428e-01 -1.07428193e+00 ...  4.06699717e-01
   4.99416508e-01  3.03965218e-01]
 [-1.73765211e-03  4.94808174e+00 -3.96642849e-04 ...  1.39673810e-04

J=11.0, eb=4.940321020899742, htf=-12.215817647041211
J=11.0, eb=4.940321020899742, htf=-12.196949068422317
J=11.0, eb=4.940321020899742, htf=-12.17842992634056
J=11.0, eb=4.940321020899742, htf=-12.160247512942838
J=11.0, eb=4.940321020899742, htf=-12.142389801228294
J=11.0, eb=4.940321020899742, htf=-12.124845397262838
J=11.0, eb=4.940321020899742, htf=-12.107603496513775
J=11.0, eb=4.940321020899742, htf=-12.090653843885436
J=11.0, eb=4.940321020899742, htf=-12.073986697085651
J=11.0, eb=4.940321020899742, htf=-12.057592792995392
J=11.0, eb=4.940321020899742, htf=-12.041463316750916
J=11.0, eb=4.940321020899742, htf=-12.025589873280024
J=11.0, eb=4.940321020899742, htf=-12.009964461062335
J=12.0, eb=4.111610872365101, htf=-16.86978290154109
J=12.0, eb=4.111610872365101, htf=-15.771170518558932
J=12.0, eb=4.111610872365101, htf=-15.260344800478883
J=12.0, eb=4.111610872365101, htf=-14.923872469543603
J=12.0, eb=4.111610872365101, htf=-14.672557946948622
J=12.0, eb=4.111610872365101, 

J=13.0, eb=3.284627203306573, htf=-14.036568802972232
J=13.0, eb=3.284627203306573, htf=-13.925343073547888
J=13.0, eb=3.284627203306573, htf=-13.825259520676775
J=13.0, eb=3.284627203306573, htf=-13.734287648156911
variance: -2.7936759973203467
integral: [[-1.38855589e+02 -2.86936512e+01  2.62500193e+02 ... -1.19812635e+01
   2.01975724e+01  1.44865119e+01]
 [-4.15048028e-03  4.43594684e+00  9.97387337e-03 ... -8.55068702e-05
   5.84867001e-04  4.19489739e-04]
 [-4.14959574e-03  4.15514245e-03  4.43110432e+00 ... -8.00104341e-04
   5.84853781e-04  4.19477366e-04]
 ...
 [ 2.03737933e-05 -3.16462696e-05 -2.19787439e-05 ...  4.43007808e+00
  -3.71389628e-06 -2.87533230e-06]
 [ 9.20543495e+00 -5.33171225e+00  1.26534577e+01 ...  1.84290248e+00
   3.32906877e+00 -1.29222129e+00]
 [ 1.18189611e+02 -2.30190496e+02 -1.39316379e+02 ...  1.80888110e+01
  -2.14377373e+01 -1.15779571e+01]]
J=13.0, eb=3.284627203306573, htf=-13.650905944903712
variance: -1.3974631406537301
integral: [[ 1.14147408e

J=14.0, eb=2.4541905752958177, htf=-12.665087169781803
J=14.0, eb=2.4541905752958177, htf=-12.635673190261167
J=14.0, eb=2.4541905752958177, htf=-12.60709972350276
J=14.0, eb=2.4541905752958177, htf=-12.579320065081323
J=14.0, eb=2.4541905752958177, htf=-12.552291298379034
J=14.0, eb=2.4541905752958177, htf=-12.525973895747283
J=14.0, eb=2.4541905752958177, htf=-12.500331370819557
J=14.0, eb=2.4541905752958177, htf=-12.475329974299745
J=14.0, eb=2.4541905752958177, htf=-12.45093842686118
J=14.0, eb=2.4541905752958177, htf=-12.427127683853048
J=14.0, eb=2.4541905752958177, htf=-12.403870727374358
J=14.0, eb=2.4541905752958177, htf=-12.38114238198237
J=14.0, eb=2.4541905752958177, htf=-12.358919150883219
J=14.0, eb=2.4541905752958177, htf=-12.337179069932363
J=14.0, eb=2.4541905752958177, htf=-12.31590157717062
J=14.0, eb=2.4541905752958177, htf=-12.29506739595331
J=14.0, eb=2.4541905752958177, htf=-12.274658430007628
J=14.0, eb=2.4541905752958177, htf=-12.254657668986473
J=14.0, eb=2.45

{'J0.0eb14.143319868324545htf-16.86978290154109': (0.06142977467703202,
  0.00960937888596116),
 'J0.0eb14.143319868324545htf-15.771170518558932': (0.1641250133781329,
  0.022864572531675575),
 'J0.0eb14.143319868324545htf-15.260344800478883': (0.24656354498817012,
  0.030961690177563297),
 'J0.0eb14.143319868324545htf-14.923872469543603': (0.31420080006916873,
  0.035912991134391455),
 'J0.0eb14.143319868324545htf-14.672557946948622': (0.37069461162195033,
  0.038879785222849655),
 'J0.0eb14.143319868324545htf-14.471887157172386': (0.41858926463077245,
  0.040561830472056526),
 'J0.0eb14.143319868324545htf-14.304832978195126': (0.4597092714314571,
  0.04139592109929455),
 'J0.0eb14.143319868324545htf-14.161732040240349': (0.4953970870557022,
  0.041662981515890485),
 'J0.0eb14.143319868324545htf-14.036568802972232': (0.5266624511713587,
  0.0415479455904163),
 'J0.0eb14.143319868324545htf-13.925343073547888': (0.5542794497434439,
  0.04117552635937016),
 'J0.0eb14.143319868324545htf-1

In [ ]:
get_correlation_times(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, sim_prms.T, N=shift_compensation.N_expr, file=file)

#### Analytical calculation for time sweep

In [8]:
for i in range(161):
    file_name = 'TimeSweep/TimeSweep_6Sites_NoPadding_NonPeriodic_SteadyState_InfiniteBath_Occupancy_2024-05-19_'+f'{i}'
    file = os.path.join(os.path.realpath('__file__'), f'../../Data/{file_name}')

    grid_prms, sim_prms, varied_prms = read_metadata(file)
    shift_compensation               = read_files_shift_compensation(file)

    get_steady_states(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, file=file)
    get_means_and_vars(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, sim_prms.T, N=shift_compensation.N_expr, limit=False, file=file)
    get_means_and_vars(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, sim_prms.T, N=shift_compensation.N_expr, limit=True, file=file)

Means could not be loaded.
Correlation times could not be loaded.
J=0.0, eb=14.143319868324545, htf=-16.0
J=0.0, eb=14.143319868324545, htf=-15.5
J=0.0, eb=14.143319868324545, htf=-15.0
J=0.0, eb=14.143319868324545, htf=-14.5
J=0.0, eb=14.143319868324545, htf=-14.0
J=0.0, eb=14.143319868324545, htf=-13.5
J=0.0, eb=14.143319868324545, htf=-13.0
J=0.0, eb=14.143319868324545, htf=-12.5
J=0.0, eb=14.143319868324545, htf=-12.0
J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.239623299753

J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.352949615690948, htf=-12.5
J=10.5, eb=5.352949615690948, htf=-12.0
J=14.0, eb=2.4541905752958177, htf=-16.0
J=14.0, eb=2.4541905752958177, htf=-15.5
J=14.0, eb=2.4541905752958177, htf=-15.0
J=14.0, eb=2.4541905752958177, htf=-14.5
J=14.0, eb=2.4541905752958177, htf=-14.0
J=14.0, eb=2.4541905752958177, htf=-13.5
J=14.0, eb=2.4541905752958177, htf=-13.0
J=14.0, eb=2.4541905752958177, htf=-12.5
J=14.0, eb=2.4541905752958177, htf=-12.0
J=0.0, eb=14.143319868324545, htf=-16.0
J=0.0, eb=14.143319868324545, htf=-15.5
J=0.0, eb=14.143319868324545, htf=-15.0
J=0.0, eb=14.143319868324545, htf=-14

J=0.0, eb=14.143319868324545, htf=-12.0
J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.35294

J=14.0, eb=2.4541905752958177, htf=-12.0
J=0.0, eb=14.143319868324545, htf=-16.0
J=0.0, eb=14.143319868324545, htf=-15.5
J=0.0, eb=14.143319868324545, htf=-15.0
J=0.0, eb=14.143319868324545, htf=-14.5
J=0.0, eb=14.143319868324545, htf=-14.0
J=0.0, eb=14.143319868324545, htf=-13.5
J=0.0, eb=14.143319868324545, htf=-13.0
J=0.0, eb=14.143319868324545, htf=-12.5
J=0.0, eb=14.143319868324545, htf=-12.0
J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8

C:\Users\tmija\AppData\Local\Temp\ipykernel_18232\1201438814.py:61: RuntimeWarning: invalid value encountered in sqrt
  stds[-1].append(np.sqrt(sigma_squared))


...

J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.352949615690948, htf=-12.5
J=10.5, eb=5.352949615690948, htf=-12.0
J=14.0, eb=2.4541905752958177, htf=-16.0
J=14.0, eb=2.4541905752958177, htf=-15.5
J=14.0, eb=2.4541905752958177, htf=-15.0
J=14.0, eb=2.4541905752958177, htf=-14.5
J=14.0, eb=2.4541905752958177, htf=-14.0
J=14.0, eb=2.

J=14.0, eb=2.4541905752958177, htf=-13.0
J=14.0, eb=2.4541905752958177, htf=-12.5
J=14.0, eb=2.4541905752958177, htf=-12.0
J=0.0, eb=14.143319868324545, htf=-16.0
J=0.0, eb=14.143319868324545, htf=-15.5
J=0.0, eb=14.143319868324545, htf=-15.0
J=0.0, eb=14.143319868324545, htf=-14.5
J=0.0, eb=14.143319868324545, htf=-14.0
J=0.0, eb=14.143319868324545, htf=-13.5
J=0.0, eb=14.143319868324545, htf=-13.0
J=0.0, eb=14.143319868324545, htf=-12.5
J=0.0, eb=14.143319868324545, htf=-12.0
J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0

J=0.0, eb=14.143319868324545, htf=-14.5
J=0.0, eb=14.143319868324545, htf=-14.0
J=0.0, eb=14.143319868324545, htf=-13.5
J=0.0, eb=14.143319868324545, htf=-13.0
J=0.0, eb=14.143319868324545, htf=-12.5
J=0.0, eb=14.143319868324545, htf=-12.0
J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.35294

J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.352949615690948, htf=-12.5
J=10.5, eb=5.352949615690948, htf=-12.0
J=14.0, eb=2.4541905752958177, htf=-16.0
J=14.0, eb=2.4541905752958177, htf=-15.5
J=14.0, eb=2.4541905752958177, htf=-15.0
J=14.0, eb=2.4541905752958177, htf=-14.5
J=14.0, eb=2.4541905752958177, htf=-14.0
J=14.0, eb=2.4541905752958177, htf=-13.5
J=14.0, eb=2.4541905752958177, htf=-13.0
J=14.0, eb=2.4541905752958177, htf=-12.5
J=14.0, eb=2.4541905752958177, htf=-12.0
Means could not be loaded.
Correlation times could not be loaded.
J=0.0, eb=14.143319868324545, htf=-16.0
J=0.0, eb=14.143319868324545, htf=-15.5
J=0.0, eb=14.143319868324545, htf=-15.0
J=0.0, eb=14.143319868324545, htf=-14.5
J=0.0, eb=14.143319868324545, htf=-14.0
J=0.0, eb=14.143319868324545, htf=-13.5
J=0.0, eb=14.143319868324545, htf=-13.0
J=0.0, eb=14.143319868324545, htf=-12.5
J=0.0

J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.352949615690948, htf=-12.5
J=10.5, eb=5.352949615690948, htf=-12.0
J=14.0, eb=2.4541905752958177, htf=-16.0
J=14.0, eb=2.4541905752958177, htf=-15.5
J=14.0, eb=2.4541905752958177, htf=-15.0
J=14.0, eb=2.4541905752958177, htf=-14.5
J=14.0, eb=2.4541905752958177, htf=-14.0
J=14.0, eb=2.4541905752958177, htf=-13.5
J=14.0, eb=2.4541905752958177, htf=-13.0
J=14.0, eb=2.4541905752958177, htf=-12.5
J=14.0, 